In [1]:
import pandas as pd
import math

In [9]:
# importo i dati
def importCSV(path):
    return pd.read_csv(path, index_col = 0)
    
df = importCSV("C:/Users/Marco/Desktop/tesi/prova/testcsv.csv")

In [10]:
# assegno True nelle caselle in cui il voto dell'esercizio è >= di 6 e False altrimenti

def replaceValueTF(df):
    dfAux = df.copy()
    flag = True
    for column in dfAux:
        dfAux.loc[dfAux[column] < 6, column] = 0
        dfAux.loc[dfAux[column] >= 6, column] = 1  
    return dfAux
        
dfTF = replaceValueTF(df)
dfTF

,Esercizio1,Esercizio2,Esercizio3
Studente,,,
Marco,1,1,0
Simone,1,0,0
Davide,1,0,1


In [37]:
# stimo l'abilità degli studenti con tecnica naif
# l'abilità è calcolata tramite media ponderata, i valori usati sono i punteggi degli studenti presi negli esercizi ed 
# i pesi sono le difficoltà assegnate agli esercizi

def getDiff(n):
    lsDiff = pd.DataFrame(columns = df.columns , index = ["Item"])
    print("Insersci la difficoltà per i seguenti item: \n")
    for columns in lsDiff:
        print(columns + ": ", end="")
        lsDiff.loc["Item",columns] = int(input())
    return lsDiff

def estimateAbility(df,lsDiff):
    lsAb = pd.DataFrame(index = df.index , columns = ["Ability"])
    for index, row in df.iterrows():
        ability = (row*(lsDiff.loc["Item"])).sum()/lsDiff.loc["Item"].sum()
        lsAb.loc[index,"Ability"] = ability
    return lsAb       

lsDiff = getDiff(len(df.columns))    
estimateAbility(df,lsDiff)

Insersci la difficoltà per i seguenti item: 

Esercizio1: 5
Esercizio2: 6
Esercizio3: 7


,Ability
Studente,
Marco,6.5
Simone,5.16667
Davide,6.83333


In [44]:
# scegliere PL per calcolare la probabilità che ogni studente risponda correttamete ad ogni item

def pl1(lsAb,lsDiff):
    dfProb = pd.DataFrame(columns=df.columns, index=df.index)
    for index, row in dfProb.iterrows():
        for columns in dfProb:
            res = (math.exp(lsAb.loc[index,"Ability"] - lsDiff.loc["Item",columns])) / (1 + math.exp(lsAb.loc[index,"Ability"] - lsDiff.loc["Item",columns]))
            dfProb.loc[index, columns] = str(round(res * 100 , 2)) + "%"
    return dfProb

def pl2():
    pass

def pl3():
    pass

pl = int(input("Scegliere il tipo di PL da utilizzare: "))

if(pl == 1):
    lsDiff = getDiff(len(df.columns))
    lsAb = estimateAbility(df,lsDiff)
    dfProb = pl1(lsAb,lsDiff)
elif(pl == 2):
    pass
elif(pl == 3):
    pass
    
dfProb

Scegliere il tipo di PL da utilizzare: 1
Insersci la difficoltà per i seguenti item: 

Esercizio1: 5
Esercizio2: 6
Esercizio3: 7


,Esercizio1,Esercizio2,Esercizio3
Studente,,,
Marco,81.76%,62.25%,37.75%
Simone,54.16%,30.29%,13.78%
Davide,86.22%,69.71%,45.84%
